In [142]:
import material_interface as mi

ni_poscar = """Ni - primitive
1.0
   2.130422000	   0.000000000	   1.230000000
   0.710140800	   2.008582000	   1.230000000
   0.000000000	   0.000000000	   2.460000000
Ni
1
Selective dynamics
cartesian
   0.000000000    0.000000000    0.000000000 T T T Ni"""
au_poscar = """Au - primitive
1.0
   2.554775000	   0.000000000	   1.475000000
   0.851591600	   2.408665000	   1.475000000
   0.000000000	   0.000000000	   2.950000000
Au
1
Selective dynamics
cartesian
   0.000000000    0.000000000    0.000000000 T T T Au"""

graphene_poscar = """Graphene
1.0
   2.467291000	   0.000000000	   0.000000000
  -1.233645000	   2.136737000	   0.000000000
   0.000000000	   0.000000000	   7.803074000
C
2
direct
   0.000000000    0.000000000    0.000000000  C
   0.333333000    0.666667000    0.000000000  C
"""

from ase.build import surface, supercells
from ase.io import read, write
from ase.visualize import view
import io
import numpy as np

ni = mi.poscar_to_atoms(ni_poscar)
gr = mi.poscar_to_atoms(graphene_poscar)

print(ni.get_cell())
print(gr.get_cell())

ni_surface = surface(ni, (1, 1, 0), layers=3)
ni_surface.center(vacuum=10, axis=2)
ni_surface.wrap()
view(ni_surface * [3, 3, 1])

gr.center(vacuum=10, axis=2)
gr.wrap()
view(gr)
# ni_slab = supercells.make_supercell(ni_surface, [[3, 0, 0], [0, 3, 0], [0, 0, 1]])
# ni_slab.center(vacuum=10, axis=2)
# ni_slab.wrap()

# view(ni_slab)

print(mi.write_atoms_to_poscar(ni))

Cell([[2.130422, 0.0, 1.23], [0.7101408, 2.008582, 1.23], [0.0, 0.0, 2.46]])
Cell([[2.467291, 0.0, 0.0], [-1.233645, 2.136737, 0.0], [0.0, 0.0, 7.803074]])
Ni 
 1.0000000000000000
     2.1304219999999998    0.0000000000000000    1.2300000000000000
     0.7101408000000000    2.0085820000000001    1.2300000000000000
     0.0000000000000000    0.0000000000000000    2.4600000000000000
 Ni 
   1
Cartesian
  0.0000000000000000  0.0000000000000000  0.0000000000000000



In [143]:
# !pip install git+https://github.com/nmdl-mizo/interface_master.git@develop

In [144]:
"""interface_master"""
from interfacemaster.cellcalc import get_primitive_hkl, get_pri_vec_inplane, get_normal_index, get_normal_from_MI, rot
from interfacemaster.interface_generator import core, convert_vector_index
from interfacemaster.symmetric_tilt import get_csl_twisted_graphenes
from numpy import array, dot, round, cross, eye, pi, ceil, column_stack, arange
from numpy.linalg import inv, det, norm
from gb_code.csl_generator import print_list
import pandas as pd

from ase.io import write
import os


def write_atoms_to_cif(atoms, name):
    temp_filename = f"{name}.cif"
    write(temp_filename, atoms, format="cif")

    with open(temp_filename, "r") as file:
        content = file.read()

    return content


ni_cif = write_atoms_to_cif(ni, "m1")
gr_cif = write_atoms_to_cif(gr, "m2")

interface = core("m1.cif", "m2.cif")

hkl_1 = get_primitive_hkl([1, 1, 1], interface.conv_lattice_1, interface.lattice_1)
hkl_2 = get_primitive_hkl([0, 0, 1], interface.conv_lattice_2, interface.lattice_2)

interface.parse_limit(du=1e-2, S=1e-2, sgm1=50, sgm2=50, dd=1e-2)

interface.search_one_position_2D(hkl_1, hkl_2, theta_range=90, dtheta=0.01, exact=False)
interface.compute_bicrystal_two_D(hkl_1, hkl_2, normal_ortho=True, lim=50, tol_ortho=1e-2, tol_integer=1e-3)

Warning!, this programme will rewrite the POSCAR file in this dir!


/Users/mat3ra/.pyenv/versions/3.11.4/lib/python3.11/site-packages/pymatgen/io/cif.py:1197: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


Congrates, we found an appx CSL!

U1 = 
[[ 0  0]
 [-4 -3]
 [ 7 -4]]; sigma_1 = 37

U2 = 
[[-3 -4]
 [ 4 -7]
 [ 0  0]]; sigma_2 = 37

D = 
[[ 0.9976693  -0.00699724  0.00420224]
 [ 0.00793899  0.99733634 -0.00349614]
 [-0.00189545  0.00512729  0.99900113]]

axis = [2.47055535 1.74694644 4.2791274 ] ; theta = 3.64

cell 1:
[[-3  0  0]
 [ 1 -4  3]
 [ 1  7  4]]
cell 2:
[[ 0 -3  4]
 [ 0  4  7]
 [-1  0  0]]


In [145]:
import material_interface as mi

settings = {
    "slab": {
        "miller:h": 1,
        "miller:k": 1,
        "miller:l": 0,
        "vacuum": 4,
        "number_of_layers": 3,
    },
    "interface": {"slab_v:matrix": [[-2, 11], [2, -11]], "layer_v:matrix": [[10, -5], [0, -20]], "distance": 2.0},
}
interface = mi.MaterialInterface(ni, gr, settings)
view(interface.structure)

LinAlgError: Singular matrix

In [231]:
"""pymatgen"""
# !pip install --upgrade pymatgen
import pymatgen
from pymatgen.analysis.interfaces.zsl import ZSLGenerator
from pymatgen.core.structure import Structure
from pymatgen.analysis.interfaces.coherent_interfaces import CoherentInterfaceBuilder

substrate = Structure.from_str(au_poscar, fmt="poscar")
layer = Structure.from_str(graphene_poscar, fmt="poscar")
layer_miller = (0, 0, 1)
substrate_miller = (1, 1, 1)

zsl = ZSLGenerator(max_area=200)
cib = CoherentInterfaceBuilder(
    substrate_structure=substrate,
    film_structure=layer,
    substrate_miller=substrate_miller,
    film_miller=layer_miller,
    zslgen=zsl,
)

cib._find_matches()
cib._find_terminations()
matches = cib.zsl_matches
terminations = cib.terminations
print("Found {} matches".format(len(matches)))

Found 73 matches


In [235]:
# print(f"MATCHES: {len(matches)}", matches)
# print(f"TERMINATIONS: {len(terminations)}", terminations)

match = matches[0]


film_sl_vectors = match.film_sl_vectors
film_vectors = match.film_vectors
substrate_sl_vectors = match.substrate_sl_vectors
substrate_vectors = match.substrate_vectors

from pymatgen.analysis.interfaces.substrate_analyzer import SubstrateMatch

substrate_match = SubstrateMatch.from_zsl(match, layer, layer_miller, substrate_miller)
print("ZSL strain:", substrate_match.strain)

film_vectors = np.vstack([film_vectors, [0, 0, 1]])
film_sl_vectors = np.vstack([film_sl_vectors, [0, 0, 1]])
substrate_vectors = np.vstack([substrate_vectors, [0, 0, 1]])
substrate_sl_vectors = np.vstack([substrate_sl_vectors, [0, 0, 1]])

from pymatgen.analysis.interfaces.zsl import rel_strain

print("strain a:", rel_strain(film_sl_vectors[0], substrate_sl_vectors[0]))
print("strain b:", rel_strain(film_sl_vectors[1], substrate_sl_vectors[1]))
print("strain c:", rel_strain(film_sl_vectors[2], substrate_sl_vectors[2]))

[[-5.15126293e-03 -9.84809863e-08  5.07879135e-17]
 [-9.84809863e-08 -5.15140543e-03 -2.56903587e-17]
 [ 5.07879135e-17 -2.56903587e-17 -5.55111512e-17]]
-0.005164561575730309
-0.005164678940099199
0.0


In [226]:
interfaces = cib.get_interfaces(terminations[0], substrate_thickness=6)
interfaces_list = list(interfaces)

from ase.visualize import view

i = interfaces_list[0].to(fmt="poscar")

structure = mi.poscar_to_atoms(i)
print(dir(structure))
a, b, c, alpha, beta, gamma = structure.cell.cellpar()

sub = mi.poscar_to_atoms(au_poscar)
lay = mi.poscar_to_atoms(graphene_poscar)
print(a, b, c)

view(mi.poscar_to_atoms(i) * [3, 3, 1])
# view(mi.poscar_to_atoms(graphene_poscar) * [3, 3, 1])
# view(mi.poscar_to_atoms(au_poscar) * [3, 3, 1])


# if len(interfaces_list) < 5:
#     for interface in interfaces_list:
#         poscar = interface.to(fmt="poscar")
#         i = mi.poscar_to_atoms(poscar)
#         view(i * [3, 3, 1])
# else:
#     poscar = interfaces_list[0].to(fmt="poscar")
#     print(poscar)
#     i = mi.poscar_to_atoms(poscar)
#     view(i * [1, 1, 1])

['__add__', '__class__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_ase_handles_dynamic_stress', '_calc', '_celldisp', '_cellobj', '_centering_as_array', '_constraints', '_del_constraints', '_get_atomic_numbers', '_get_constraints', '_get_positions', '_masked_rotate', '_pbc', '_set_positions', 'append', 'arrays', 'ase_objtype', 'calc', 'cell', 'center', 'constraints', 'copy', 'edit', 'euler_rotate', 'extend', 'fromdict', 'get_all_distances', 'get_angle', 'get_angles', 'get_angular_momentum', 'get_array', 'get_atomic_numbers', 'get_calculator', 'get_cell', 'get_cell_lengths_and_angles', 'get_celldisp

<Popen: returncode: None args: ['/Users/mat3ra/.pyenv/versions/3.11.4/bin/py...>

2023-10-05 15:06:46.685 python[98255:3391774] *** CFMessagePort: bootstrap_register(): failed 1100 (0x44c) 'Permission denied', port = 0x14d07, name = 'python.ServiceProvider'
See /usr/include/servers/bootstrap_defs.h for the error codes.
